<a href="https://colab.research.google.com/github/Rajathu17/Dual-Factor-Quantum-CAPTCHA/blob/main/Hybrid_Quantum_CAPTCHA_DUAL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Step 1: Install Dependencies

In [9]:
import subprocess, sys

def install(package):
    subprocess.check_call([sys.executable, "-m", "pip", "install", package])

try:
    import qiskit
    from qiskit import QuantumCircuit, transpile
    from qiskit.providers.basic_provider import BasicProvider
except ImportError:
    print("Installing Qiskit & dependencies...")
    install('qiskit')
    install('pillow')
    install('ipywidgets')
    print("Done. Please re-run the cell.")


# Step 2: Imports and Configuration

In [10]:
import math, random, secrets, io
from typing import List, Dict
from PIL import Image, ImageDraw, ImageFont, ImageFilter
import ipywidgets as widgets
from IPython.display import display, clear_output

CHARSET = 'ABCDEFGHJKLMNPQRSTUVWXYZ23456789'


# Step 3: Quantum Random Number Generator

In [11]:
from qiskit import QuantumCircuit, transpile
from qiskit.providers.basic_provider import BasicProvider

class QuantumRNG:
    def __init__(self):
        try:
            self.backend = BasicProvider().get_backend('basic_simulator')
            self.available = True
        except:
            self.available = False

    def get_bits(self, n_bits: int):
        if not self.available:
            import secrets
            return [secrets.randbits(1) for _ in range(n_bits)]
        try:
            qc = QuantumCircuit(n_bits)
            for i in range(n_bits):
                qc.h(i)
            qc.measure_all()
            job = self.backend.run(transpile(qc, self.backend), shots=1)
            bits = [int(b) for b in list(job.result().get_counts().keys())[0]]
            if len(bits) < n_bits:
                bits = [0]*(n_bits-len(bits)) + bits
            return bits[:n_bits]
        except:
            import secrets
            return [secrets.randbits(1) for _ in range(n_bits)]


# Step 4: Hybrid CAPTCHA Engine (Text + Geometry)

In [12]:
class HybridEngine:
    SHAPE_MAP = {0:3,1:4,2:5,3:6,4:8,5:3,6:4,7:10}
    SHAPE_NAMES = {3:'Triangle',4:'Square',5:'Pentagon',6:'Hexagon',8:'Octagon',10:'Decagon'}

    @staticmethod
    def bits_to_text(bits, length=4):
        seed_text=""
        i=0
        while len(seed_text)<length:
            chunk=0
            for _ in range(5):
                bit = bits[i] if i<len(bits) else random.getrandbits(1)
                chunk = (chunk<<1)|bit
                i+=1
            seed_text += CHARSET[chunk % len(CHARSET)]
        return seed_text

    @staticmethod
    def get_shape_params(bits):
        shape_val = int("".join(map(str,bits[:3])),2)
        sides = HybridEngine.SHAPE_MAP.get(shape_val,4)
        rot_val = int("".join(map(str,bits[3:11])),2)
        angle = math.radians((rot_val/255)*360)
        return {'sides':sides,'angle':angle,'noise':bits[11:],'name':HybridEngine.SHAPE_NAMES.get(sides,'Polygon')}

    @staticmethod
    def generate_hybrid_captcha(shape_params, text, width=180, height=180):
        img = Image.new("RGB",(width,height),(255,255,255))
        draw = ImageDraw.Draw(img)
        center=(width//2,height//2)
        radius=min(width,height)//2.5
        vertices=[]
        noise_bits=shape_params['noise']
        bit_idx=0
        for i in range(shape_params['sides']):
            theta = (2*math.pi*i)/shape_params['sides'] + shape_params['angle']
            x = center[0] + radius*math.cos(theta)
            y = center[1] + radius*math.sin(theta)
            if bit_idx+2 < len(noise_bits):
                x += (noise_bits[bit_idx]*2 -1)*4
                y += (noise_bits[bit_idx+1]*2 -1)*4
                bit_idx+=2
            vertices.append((x,y))
        draw.polygon(vertices, fill="#2c3e50", outline="#34495e")
        try: font = ImageFont.truetype("arial.ttf",32)
        except: font=ImageFont.load_default()
        bbox=draw.textbbox((0,0),text,font=font)
        tx=center[0]-(bbox[2]-bbox[0])//2
        ty=center[1]-(bbox[3]-bbox[1])//2
        for ch in text:
            ox=random.randint(-3,3); oy=random.randint(-3,3)
            draw.text((tx+ox,ty+oy),ch,font=font,fill=(255,255,255))
            tx+=(bbox[2]-bbox[0])//len(text)+2
        for _ in range(5):
            x1,y1=random.randint(0,width),random.randint(0,height)
            x2,y2=random.randint(0,width),random.randint(0,height)
            draw.line((x1,y1,x2,y2),fill=(200,200,200),width=1)
        import io
        img=img.filter(ImageFilter.GaussianBlur(0.5))
        buf=io.BytesIO(); img.save(buf, format="PNG")
        return buf.getvalue()


# Step 5: UI Application

In [13]:
class HybridCaptchaApp:
    def __init__(self):
        self.rng=QuantumRNG()
        self.output=widgets.Output()
        self.target_data={}
        self.status=widgets.Label(value="Initializing...")
        self.instruction=widgets.HTML("<h3>...</h3>")
        self.btn_refresh=widgets.Button(description="New Challenge")
        self.btn_refresh.on_click(self.refresh)
        self.img_widgets=[]
        self.selection_wrappers=[]
        for i in range(3):
            img_w=widgets.Image(format='png',width=180,height=180)
            btn=widgets.Button(description="Select")
            btn.on_click(lambda b, idx=i: self.verify(idx))
            self.img_widgets.append(img_w)
            self.selection_wrappers.append(widgets.VBox([img_w,btn]))
        self.ui=widgets.VBox([self.status,self.instruction,widgets.HBox(self.selection_wrappers),self.btn_refresh,self.output])

    def refresh(self,_=None):
        self.output.clear_output()
        self.status.value="Consulting Quantum Oracle..."
        bits=self.rng.get_bits(60)
        shape_bits=bits[:20]; text_bits=bits[20:50]
        t_shape=HybridEngine.get_shape_params(shape_bits)
        t_text=HybridEngine.bits_to_text(text_bits,length=3)
        self.target_data={'shape':t_shape,'text':t_text}
        options=[{'data':self.target_data,'is_target':True}]
        import random
        d1_shape=HybridEngine.get_shape_params([random.randint(0,1) for _ in range(20)])
        while d1_shape['sides']==t_shape['sides']:
            d1_shape=HybridEngine.get_shape_params([random.randint(0,1) for _ in range(20)])
        options.append({'data':{'shape':d1_shape,'text':t_text},'is_target':False})
        d2_text=HybridEngine.bits_to_text([random.randint(0,1) for _ in range(30)],3)
        options.append({'data':{'shape':t_shape,'text':d2_text},'is_target':False})
        random.shuffle(options)
        self.current_options=options
        self.instruction.value=f"Click the <b>{t_shape['name']}</b> containing <b>{t_text}</b>"
        for i,opt in enumerate(options):
            self.img_widgets[i].value = HybridEngine.generate_hybrid_captcha(opt['data']['shape'],opt['data']['text'])
        self.status.value="Ready."

    def verify(self,idx):
        with self.output:
            clear_output()
            if self.current_options[idx]['is_target']:
                print("ACCESS GRANTED")
            else:
                print("ACCESS DENIED")


# Step 6: Run the Application

In [14]:
app=HybridCaptchaApp()
display(app.ui)
app.refresh()